<a href="https://www.kaggle.com/code/handeatasagun/hyperparam-opt-ensemble-learning-10-ml-models?scriptVersionId=144760752" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# From Vine to Palate: 🍷
# Wine Quality Prediction through Machine Learning Models

## Business Problem

The key challenge in the wine industry is predicting people's wine taste preferences in advance. This is crucial for making informed decisions in wine production, marketing, and creating targeted consumer strategies. This problem can be addressed by analyzing a substantial dataset linked to physicochemical and sensory tests during the certification phase. Such analyses can help wine producers understand taste preferences, improve production processes, and inform niche market marketing strategies. This business issue can be resolved through machine learning techniques.

## Dataset Story

The dataset pertains to various types of Portuguese 'Vinho Verde' red wines. For more information, you can refer to the study by Cortez et al.,titled 'Modeling wine preferences by data mining from physicochemical properties' (2009). Due to privacy and logistical constraints, only physicochemical variables are available (there is no data on factors such as grape types, wine brands, wine sale prices, etc.)

The dataset consists of the following features:
- fixed acidity
- volatile acidity
- citric acid
- residual sugar
- chlorides
- free sulfur dioxide
- total sulfur dioxide
- density
- pH
- sulphates
- alcohol

The output variable, which we aim to predict, is the wine's __quality__, scored between 0 and 10.

# 1. Importing Required Libraries and Datasets

## 1.1. Importing libraries:

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.neighbors import LocalOutlierFactor
from sklearn.preprocessing import StandardScaler
from IPython.display import display, HTML

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.model_selection import cross_validate, cross_val_predict
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import VotingClassifier
from itertools import combinations

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.set_option('display.width', 500)

import warnings
warnings.simplefilter("ignore")

## 1.2. Reading dataset:

In [ ]:
df = pd.read_csv('/kaggle/input/red-wine-quality-cortez-et-al-2009/winequality-red.csv')

# 2. Exploratory Data Analysis (EDA)

## 2.1. Data frame inspection function:

This function is designed to inspect and provide a summary of the essential characteristics and information of a dataframe.

In [ ]:
def check_df(dataframe, head=5):
    display(HTML(f"<h3>Types</h3>{dataframe.dtypes.to_frame().to_html()}"))
    display(HTML(f"<h3>Head</h3>{dataframe.head(head).to_html()}"))
    display(HTML(f"<h3>Shape</h3>{dataframe.shape}"))
    display(HTML(f"<h3>NA</h3>{dataframe.isnull().sum().to_frame().to_html()}"))
    display(HTML(f"<h3>Quantiles</h3>{dataframe.describe([0.25, 0.50, 0.95]).T.to_html()}"))

    
check_df(df)

- The dataset consists of 1599 rows and 12 columns. 
- All variables are numerical. 
- There are no missing values in any of the columns. 
- It is suspected that there may be outliers in some of the variables.

## 2.2. Detecting dataframe column types function:
This function analyzes the columns in a dataframe and determines categorical, numerical, and other columns.

Parameters:
- cat_th (int): Threshold value for considering a column as categorical.
- car_th (int): Threshold value for considering a column as having high cardinality.

In [ ]:
def grab_col_names(dataframe, cat_th=10, car_th=20):

    # cat_cols, cat_but_car
    cat_cols = [col for col in dataframe.columns if dataframe[col].dtypes == "O"]
    num_but_cat = [col for col in dataframe.columns if dataframe[col].nunique() < cat_th and
                   dataframe[col].dtypes != "O"]
    cat_but_car = [col for col in dataframe.columns if dataframe[col].nunique() >= car_th and
                   dataframe[col].dtypes == "O"]
    cat_cols = cat_cols + num_but_cat
    cat_cols = [col for col in cat_cols if col not in cat_but_car]

    # num_cols
    num_cols = [col for col in dataframe.columns if dataframe[col].dtypes != "O"]
    num_cols = [col for col in num_cols if col not in num_but_cat]

    print(f"Observations: {dataframe.shape[0]}")
    print(f"Variables: {dataframe.shape[1]}")
    print(f'cat_cols: {len(cat_cols)}')
    print(f'num_cols: {len(num_cols)}')
    print(f'cat_but_car: {len(cat_but_car)}')
    print(f'num_but_cat: {len(num_but_cat)}')
    
    return cat_cols, num_cols, cat_but_car


cat_cols, num_cols, cat_but_car = grab_col_names(df)

- Out of the 12 variables in the dataset, 11 are numerical variables, while 1 variable is numerical but actually categorical ("quality').

## 2.3. Categorical column summary function:

The function visualizes the frequency and percentage of a categorical column.


In [ ]:
def cat_summary(dataframe, col_name, plot=False):
    if plot:
        sns.set_palette("Set2")
        plt.figure(figsize=(8, 4))
        ax = sns.countplot(y=col_name, data=dataframe, order=dataframe[col_name].value_counts().index)
        
        for p in ax.patches:
            width = p.get_width()
            ax.annotate(f'{width / len(dataframe) * 100:.1f}%', (width, p.get_y() + p.get_height() / 2.),
                        ha='left', va='center')
        
        plt.show(block=True)
        

for col in cat_cols:
    cat_summary(df, col, plot=True)

- The values of the "quality" variable are mostly 5 or 6. This indicates that the examined wines are of **medium** quality.

## 2.4. Numerical column summary function:

The function visualizes the frequency and percentage of a numerical column.


In [ ]:
def num_summary(dataframe, numerical_col, plot=False):
    quantiles = [0.05, 0.10, 0.20, 0.30, 0.40, 0.50, 0.60, 0.70, 0.80, 0.90, 0.95, 0.99]
    summary = dataframe[numerical_col].describe(quantiles)

    if plot:
        fig, axes = plt.subplots(1, 2, figsize=(18, 4))
        
        # Plot histogram
        dataframe[numerical_col].hist(bins=20, ax=axes[0], color="lightblue")
        axes[0].set_xlabel(numerical_col)
        axes[0].set_title(numerical_col)
        
        # Display summary statistics as text
        summary_text = "\n".join([f'{col}: {value:.3f}' for col, value in summary.items()])
        axes[1].text(0.5, 0.5, summary_text, fontsize=12, va='center', ha='left', linespacing=1.5)
        axes[1].axis('off')  # Hide axis for the summary table
        
        plt.show()
        

for col in num_cols:
    num_summary(df, col, plot = True)

- Except for "__density__" and "__pH__," it is observed that all other variables in the dataset are generally __right-skewed__.

## 2.5. Outlier detection:

In this section, we will try to identify outliers in each dataframe and take necessary actions if they exist. 

In [ ]:
# The function is used to calculate the outlier thresholds for a specific column in a dataframe.
################################################################
def outlier_thresholds(dataframe, col_name, q1=0.05, q3=0.95):
    quartile1 = dataframe[col_name].quantile(q1)
    quartile3 = dataframe[col_name].quantile(q3)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range
    return low_limit, up_limit


# This function is designed to check for outliers.
################################################################
def check_outlier(dataframe, col_name):
    low_limit, up_limit = outlier_thresholds(dataframe, col_name)
    if dataframe[(dataframe[col_name] > up_limit) | (dataframe[col_name] < low_limit)].any(axis=None):
        return True
    else:
        return False

    
# The function is designed to identify and retrieve outlier data points in a specified column of a dataframe 
# based on predefined outlier thresholds.
################################################################
def grab_outliers(dataframe, col_name, index=False):
    low, up = outlier_thresholds(dataframe, col_name)

    outlier_df = dataframe[((dataframe[col_name] < low) | (dataframe[col_name] > up))]

    if outlier_df.shape[0] > 10:
        display(outlier_df.head())
    else:
        display(outlier_df)

    if index:
        outlier_index = outlier_df.index
        return outlier_index

    return outlier_df.shape[0]

In [ ]:
for col in num_cols:
    print(col, check_outlier(df, col))

- When examining the dataset for outliers, it was observed that the variables "__residual sugar__", "__chlorides__", "__total sulfur dioxide__", and "__sulphates__" contain outliers.

In [ ]:
for col in num_cols:
    print(f"Outliers in {col}:")
    grab_outliers(df, col)
    print("\n")

- There are a few outliers in the "__total sulfur dioxide__" and "__sulphates__" variables, while there are more than 10 outliers in the "__residual sugar__" and "__chlorides__" variables.
- Outliers will be processed using the __Local Outlier Factor (LOF) method__. It is an outlier detection algorithm that identifies unusual or isolated data points based on their local densities, with points having high LOF values typically considered as outliers.

In [ ]:
# Detecting Outliers with Local Outlier Factor (LOF) method
###########################################################

# Initialize the LOF model and consider 20 neighbors for each data point
clf = LocalOutlierFactor(n_neighbors=20)
clf.fit_predict(df)

# Get the LOF scores
df_scores = clf.negative_outlier_factor_

# Transfer LOF scores into a sorted dataframe
scores = pd.DataFrame(np.sort(df_scores))

# Create a plot showing LOF scores within the first 50 data points
scores.plot(stacked=True, xlim=[0, 50], style='.-')
plt.show()

# Set a threshold for detecting outliers
th = np.sort(df_scores)[4]

# Remove outliers
rows_to_drop = df[df_scores < th].index
df.drop(axis=0, index=rows_to_drop, inplace=True)
df.shape

- Based on the elbow method, it was determined that there are __4 outliers__, and these rows have been __removed__ from the dataset.

## 2.6. Detecting missing values function:

The function generates a table summarizing missing values in the given dataframe.

In [ ]:
def missing_values_table(dataframe, na_name=False):
    na_columns = [col for col in dataframe.columns if dataframe[col].isnull().sum() > 0]

    n_miss = dataframe[na_columns].isnull().sum().sort_values(ascending=False)
    ratio = (dataframe[na_columns].isnull().sum() / dataframe.shape[0] * 100).sort_values(ascending=False)
    missing_df = pd.concat([n_miss, np.round(ratio, 2)], axis=1, keys=['n_miss', 'ratio'])
    print(missing_df, end="\n")

    if na_name:
        return na_columns

missing_values_table(df)

- As previously mentioned, there are no missing values in the dataset.

## 2.7. Correlation heatmap function:

This function plots a correlation heatmap for the given dataframe.

In [ ]:
def plot_correlation_heatmap(dataframe, threshold=0.5, title="Correlation Heatmap"):
    corr = dataframe.corr()
    high_corr = corr[abs(corr) >= threshold]
    mask = np.triu(np.ones_like(high_corr, dtype=bool))
    fig, ax = plt.subplots(figsize=(8, 6))
    vmin = -1.0
    vmax = 1.0
    sns.heatmap(high_corr, annot=True, fmt=".2f", cmap='coolwarm', mask=mask, 
                linewidths=0.5, ax=ax, vmin=vmin, vmax=vmax, cbar_kws={'label': 'Correlation'})
    ax.set_title(title)
    plt.show()
    
    
plot_correlation_heatmap(df)

- An increase in citric acid is associated with an increase in fixed acidity and a decrease in volatile acidity.
- As total sulfur dioxide increases, free sulfur dioxide also increases.
- Fixed acidity has a positive correlation with density and a negative correlation with pH. Similarly, citric acid has a negative correlation with pH.
- Since the correlation coefficients are not very high, there are no strong relationships between variables. Therefore, no variable has been removed from the dataset.

# 3. Feature Engineering

## 3.1. Creating new features:

In [ ]:
df['_alcohol_x_sulphates_'] = df['alcohol'] * df['sulphates']

df['_alcohol_density_ratio_'] = df['alcohol'] / df['density']

df.head()

- Before creating these variables, models were run without adding any variables, and the top 5 features were determined according to feature importance. Using these features, these two variables were created.

## 3.2. Encoding processes:

This process is done to convert categorical variables into numerical ones. In fact, the "__quality__" categorical variable in this dataset is already expressed numerically. However, this section has been added to perform the necessary operations on any __rare groups__ in this variable.

In [ ]:
def rare_analyser(dataframe, target, cat_cols):
    for col in cat_cols:
        print(col, ":", len(dataframe[col].value_counts()))
        print(pd.DataFrame({"COUNT": dataframe[col].value_counts(),
                            "RATIO": dataframe[col].value_counts() / len(dataframe),
                            "TARGET_MEAN": dataframe.groupby(col)[target].mean()}), end="\n\n\n")

rare_analyser(df, "quality", cat_cols)

- As a result of the rare analysis, it was observed that some groups had very low percentages. 
- Based on this, a decision was made to classify the groups that received __3, 4, and 5__ points as '__0__", and those that received __6, 7, and 8__ points as '__1__', thereby categorizing wine quality only as 'good' and 'bad'.

In [ ]:
df["quality"] = np.where(df["quality"] > 5, 1, 0)
df["quality"].head()

## 3.3. Standardization:

The process is to make variables with different scales or distributions have the same scale or distribution.

In [ ]:
scaler = StandardScaler()

num_cols = [col for col in df.columns if col != 'quality']

df[num_cols] = scaler.fit_transform(df[num_cols])

df.head()

# 4. Modelling

In this section, __10__ different ML models will be evaluated suitable for a classification problem. __Hyperparameter optimization__ will be applied to these models, and __ensemble learning__ will be performed for the models that yield the best results.

In [ ]:
X = df.drop("quality", axis=1)
y = df['quality']

In [ ]:
check_df(X)  # The independent variables have been checked.

## 4.1. Base models evaluation function:

The __evaluate_model function__ assesses the performance of a given machine learning model using cross-validation on a dataset, calculating key classification metrics such as __accuracy__, __F1 score__, and __ROC AUC__. The results are aggregated into a dataframe and sorted to identify the best-performing models.

In [ ]:
def evaluate_model(model, X, y):
    cv_results = cross_validate(model, X, y, cv=5, scoring=["accuracy", "f1", "roc_auc"])
    report = classification_report(y, cross_val_predict(model, X, y, cv=5))
    mean_accuracy = cv_results['test_accuracy'].mean()
    mean_f1 = cv_results['test_f1'].mean()
    mean_roc_auc = cv_results['test_roc_auc'].mean()
    return {
        "report": report,
        "accuracy_mean": mean_accuracy,
        "f1_mean": mean_f1,
        "roc_auc_mean": mean_roc_auc
    }

models = [('KNN', KNeighborsClassifier()),
    ('LR', LogisticRegression(random_state=17)),
    ("CART", DecisionTreeClassifier(random_state=17)),
    ("SVC", SVC(random_state=17)),
    ("RF", RandomForestClassifier(random_state=17)),
    ('Adaboost', AdaBoostClassifier(random_state=17)),
    ('GBM', GradientBoostingClassifier(random_state=17)),
    ('LGBM', LGBMClassifier(random_state=17)),
    ('CatBoost', CatBoostClassifier(random_state=17, verbose=False)),
    ('XGBoost', XGBClassifier(random_state=17))
]


results_df = pd.DataFrame(columns=["Model", "Accuracy", "F1 Score", "ROC AUC"])

results = []

for model_name, model in models:
    model_results = evaluate_model(model, X, y)
    results.append({
        "Model": model_name,
        "Accuracy": model_results["accuracy_mean"],
        "F1 Score": model_results["f1_mean"],
        "ROC AUC": model_results["roc_auc_mean"]
    })

results_df = pd.DataFrame(results)
results_df = results_df.sort_values(by="Accuracy", ascending=False)
print(results_df)

- When we rank the models based on accuracy, we observe that the top 3 models are **CatBoost**, **LR** and **SVC**.
- The models whose accuracy falls **below 0.7** are **KNN**, and **CART**.

## 4.2. Feature importance function:

This function calculates and plots the importance scores of the features, helping to identify which features have the most significant impact on the model's predictions. 

In [ ]:
def plot_importance(model, features, num=len(X)):
    feature_imp = pd.DataFrame({'Value': model.feature_importances_,
                                'Feature': features.columns})
    
    plt.figure(figsize=(8, 4))
    sns.set(font_scale=1)
    sns.barplot(x="Value", y="Feature",
                data=feature_imp.sort_values(by="Value",
                                             ascending=False)[0:num])
    plt.title('Features')
    plt.tight_layout()
    plt.show()


# CatBoost -> high accuracy
catboost_model = models[8][1]
catboost_model.fit(X, y)

plot_importance(catboost_model, X)


- The fact that the new variables resulting from the interaction of variables with each other are among the top 3 most important variables suggests that these variables make a significant contribution to the model's performance.

## 4.3. Automated hyperparameter optimization:

Hyperparameter optimization was performed on machine learning models to achieve their best performance. This process was automated using the '**hyperparameter_optimization function**'.

In [ ]:
# Hyperparameter grids:
#######################

knn_params = {"n_neighbors": range(2, 50)}

cart_params = {'max_depth': range(1, 20),
               "min_samples_split": range(2, 30)}

rf_params = {"max_depth": [10, 12, 15],
             "max_features": [0.5, 1, 2],
             "min_samples_split": [12, 15, 17],
             "n_estimators": [310, 320, 330]}

xgboost_params = {"learning_rate": [0.035, 0.03, 0.025],
                  "max_depth": [3, 4, 5],
                  "n_estimators": [190, 200, 210]}

lightgbm_params = {"learning_rate": [0.004, 0.005, 0.006],
                   "n_estimators": [280, 300, 320]}

svc_params = {"C": [1.1, 1.2, 1.3],
              "kernel": ["linear", "rbf"]}

lr_params = {"C": [0.13, 0.14, 0.15],
             "solver": ["liblinear", "lbfgs"]}

adaboost_params = {"n_estimators": [70, 75, 80],
                   "learning_rate": [0.1, 0.11, 0.12]}

gbm_params = {"n_estimators": [14, 15, 16],
              "learning_rate": [0.04, 0.05, 0.06],
              "max_depth": [1, 2, 3]}

catboost_params = {"iterations": [450, 460, 470],
                   "learning_rate": [0.005, 0.006, 0.007],
                   "depth": [2, 3, 4]}

In [ ]:
# Classification algorithms:
############################

classifiers = [('KNN', KNeighborsClassifier(), knn_params),
    ("CART", DecisionTreeClassifier(random_state=17), cart_params),
    ("RF", RandomForestClassifier(random_state=17), rf_params),
    ('XGBoost', XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=17), xgboost_params),
    ('LightGBM', LGBMClassifier(random_state=17), lightgbm_params),
    ('SVC', SVC(random_state=17, probability=True), svc_params),
    ('LR', LogisticRegression(random_state=17), lr_params),
    ('Adaboost', AdaBoostClassifier(random_state=17), adaboost_params),
    ('GBM', GradientBoostingClassifier(random_state=17), gbm_params),
    ('CatBoost', CatBoostClassifier(verbose=False, random_state=17), catboost_params),
]

In [ ]:
def hyperparameter_optimization(X, y, cv=5, scoring="accuracy"):
    print("Hyperparameter Optimization....")
    best_models = {}
    for name, classifier, params in classifiers:
        print(f"########## {name} ##########")
        cv_results = cross_validate(classifier, X, y, cv=cv, scoring=scoring)
        print(f"{scoring} (Before): {round(cv_results['test_score'].mean(), 4)}")

        gs_best = GridSearchCV(classifier, params, cv=cv, n_jobs=-1, verbose=False).fit(X, y)
        final_model = classifier.set_params(**gs_best.best_params_)

        cv_results = cross_validate(final_model, X, y, cv=cv, scoring=scoring)
        print(f"{scoring} (After): {round(cv_results['test_score'].mean(), 4)}")
        print(f"{name} best params: {gs_best.best_params_}", end="\n\n")
        best_models[name] = final_model
    return best_models

best_models = hyperparameter_optimization(X, y)

- The optimization process has resulted in expected improvements in the performance of all models, with all models achieving an accuracy value exceeding 0.7 except for CART. It has been observed that the model with the highest accuracy is __CatBoost (0.7561)__.

## 4.4. Ensemble learning:

We apply ensemble learning to combine multiple machine learning models to improve the overall performance of the model and achieve better predictions. All models were tested in triple combinations, and the combination that yielded the highest accuracy value was determined.

In [ ]:
best_accuracy = 0
best_combination = None
best_classifiers = None

combination_sizes = [3]  # [2, 3, 4, ...] combinations of 2 or more models can be tried

for size in combination_sizes:
    for combination in combinations([name for name, _, _ in classifiers], size):
        combination_name = "_".join(combination)
        selected_classifiers = [(name, model, params) for name, model, params in classifiers if name in combination]

        combined_estimators = [(name, model) for name, model, _ in selected_classifiers]
        combined_classifier = VotingClassifier(estimators=combined_estimators, voting='soft')

        cv_results = cross_validate(combined_classifier, X, y, cv=5, scoring="accuracy")
        current_accuracy = cv_results['test_score'].mean()

        if current_accuracy > best_accuracy:
            best_accuracy = current_accuracy
            best_combination = combination_name
            best_classifiers = selected_classifiers

print(f"Best Combination: {best_combination}")
print(f"Best Accuracy: {round(best_accuracy, 4)}")


# VotingClassifier
best_estimators = [(name, model) for name, model, _ in best_classifiers]
best_voting_clf = VotingClassifier(estimators=best_estimators, voting='soft')
best_voting_clf.fit(X, y)


It has been observed that the combination of the models __Adaboost__, __GBM__, and __CatBoost__ yielded the best result among the used models, achieving a higher accuracy value (__0.7567__) compared to using the models individually.

# 5. Conclusion

In this study, it has been observed that the combination of "__Adaboost, CatBoost, and GBM__" determined through ensemble learning yielded the highest accuracy value. It has been decided to use this combination for predicting red wine quality.

## If you find this notebook useful then please upvote. Thank you in advance.